# Integration Magics-Matplotlib-cartopy


Install ecmwlibs ( to get magics C++ library)
pip install ecmwflibs==0.4.19rc1
Install magics-python locally
pip install -e .

In [22]:
import pyproj
from Magics import macro as magics
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

In [23]:
magics.version()

'Magics 4.13.0'

# Classic plot using magics-python + Magics C++library 

May need to install ecmwflibs first   
pip install ecmwlibs 

Here we create a temporary png file, and we retun an Image to display


In [24]:

style = {'description_width': 'initial', 'width': 'max-content'}

@interact(
    map_coastline_resolution = widgets.Dropdown(description='map_coastline_resolution', 
                                                 options = ["medium", "high", "low"], value='medium', 
                                                 style=style),
    map_coastline_colour = widgets.Dropdown(description='map_coastline_colour', 
                                                 options = ["navy", "evergreen", "charcoal"], value='navy', 
                                                 style=style),
)
def plot(**kwargs):
    coastlines = magics.mcoast(kwargs)
    return magics.plot(coastlines)
    

    

interactive(children=(Dropdown(description='map_coastline_resolution', options=('medium', 'high', 'low'), styl…


# Integration with cartopy and matplotlib 

We ask MagicsC++ to create an output in *binary format*.  
We setup a matplotlib figure, and a projection using cartopy 
We use the binary reader to read the output, and convert  to matplotlib primitives 
WE ask cartopy to plot the coastlines. 
We can also add some other elements such as points on the plots. 
But the legend has to be fixed 


In [35]:
import cartopy.crs as ccrs
style = {'description_width': 'initial', 'width': 'max-content'}

projections = {
        "mollweide": ccrs.Mollweide(),
        "robinson" : ccrs.Robinson(),
        "cylindrical" : ccrs.PlateCarree()
    }
@interact(
    projection = widgets.Dropdown(description='Projection', 
                                  options = projections.keys(), 
                                  style=style),
    domain = widgets.RadioButtons(options=['global', 'sub-area'], 
                                  description='Domain',
                                  style=style),
    legend = widgets.RadioButtons(options=['on', 'off'], 
                                  value="off",
                                  description='Legend',
                                  style=style),
)
def plot(projection, domain, legend):
    area = magics.mmap(
      subpage_map_projection = projection,
    )
    
    data = magics.mgrib(grib_input_file_name = "t850.grib")
    contour = magics.mcont(
        legend                       = legend,
        contour                      = 'off',
        contour_shade                = 'on',
        contour_label                = 'off',
        contour_level_selection_type = "interval",
        contour_interval             = 2.0,
        contour_shade_method         = 'area_fill',
        contour_shade_colour_method  = "calculate",
        contour_shade_max_level_colour = "red",
        contour_shade_min_level_colour = "blue",
        contour_shade_colour_direction = "clockwise"
        )
        
    coastlines = magics.mcoast()
    legend = magics.mlegend(legend_display_type="continuous" , 
                            legend_label_frequency=1, 
                            legend_entry_border = False)
    
    coastlines = magics.mcoast(map_coastline_colour="grey")
    fig = plt.figure(figsize=(20,20))
    
    axis = fig.add_subplot(1,1,1, projection=projections[projection])
    magics.matplotlib_plot(area, data, contour, axes=axis)
    axis.coastlines(color="black", zorder=100)
    axis.gridlines(color="black", draw_labels=True, zorder=101)
    plt.title("Magics and Cartopy")
    axis.scatter(-1,52, 200, transform=ccrs.Geodetic())
    if  domain == "sub-area" :
        axis.set_extent((-25, 45, 32, 72))
    

interactive(children=(Dropdown(description='Projection', options=('mollweide', 'robinson', 'cylindrical'), sty…